In [82]:
import dash
import dash_mantine_components as dmc
from dash import html, dcc, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px

import webbrowser
import pandas as pd
import numpy as np

from statistics import variance, stdev

In [2]:
df = pd.read_csv("combined_processed.csv")

In [3]:
castle_heroes = ['Adelaide', 'Orrin', 'Valeska', 'Edric', 'Sylvia', 
                 'Beatrice', 'Lord Haart', 'Sorsha', 'Christian', 
                 'Tyris', 'Rion', 'Adela', 'Cuthbert', 'cuttbert','Ingham', 'Sanya', 'Loynis', 
                 'Caitlin', 'Katarzyna', 'Roland']

rampart_heroes = ['Mephala', 'Ufretin', 'Jenova', 'Ryland', 'Giselle', 'Ivor', 'Clancy', 'Thorgrim', 'Kyrre', 
                  'Coronius', 'Uland', 'Elleshar', 'Gem', 'Malcom', 'Melodia', 'Alagar', 'Aeris']

tower_heroes = ['Piquedram', 'Thane', 'Josephine', 'Neela', 'Torosar', 'Fafner', 'Rissa', 'Iona', 'Astral', 
                'Halon', 'Serena', 'Daremyth','Theodorus', 'Solmyr', 'Cyra', 'Aine', 'Dracon']

inferno_heroes = ['Fiona', 'Rashka', 'Marius', 'Ignatius', 'Octavia', 'Calh', 
                  'Pyre', 'Nymus', 'Ayden', 'Xyron', 'Axsis', 'Olema','Calid', 'Ash', 'Zydar', 'Xarfax']

necropolis_heroes = ['Straker', 'Vokial', 'Moandor', 'Charna','Tamika', 'Isra', 'Clavius', 'Ranloo', 
                     'Septienna', 'Aislinn', 'Sandro', 'Nimbus', 'Thant', 'Xsi', 'Vidomina', 'Nagash',
                    'Haart Lich', 'Galthran']

dungeon_heroes = ['Lorelei', 'lorelai', 'Arlach', 'Dace', 'Ajit', 'Damacon', 'Gunnar', 
                  'Synca', 'Shakti', 'Alamar', 'Jaegar','Malekith', 'Jeddite', 
                  'Deemer', 'Geon', 'Sephinroth', 'Darkstorn', 'Mutare Drake', 'Mutare']

stronghold_heroes = ['Yog', 'Gurnisson', 'Jabarkas', 'Crag Hack', 'Shiva', 'Gretchin', 'Krellion', 
                     'Tyraxor', 'Gird', 'Vey', 'Dessa', 'Terek', 'Zubin', 'Gundula', 'Oris', 'Saurug', 'Boragus']

fortress_heroes = ['Bron', 'Drakon', 'Wystan', 'Tazar', 'Alkin', 'Korbac', 'Gerwulf', 'Broghild', 
                   'Mirlanda', 'Rosic', 'Voy', 'Verdish', 'Kinkeria', 'Merist', 'Styg', 'Andra', 'Tiva']

conflux_heroes = ['Pasis', 'Thunar', 'Ignissa', 'Lacus', 'Kalt', 'Fiur', 'Erdamon', 'Monere', 
                  'Luna', 'Inteus', 'Grindan', 'Labetha', 'Ciele', 'Gelare', 'Aenain', 'Brissa']

cove_heroes = ['Cassiopeia', 'Derek', 'Anabel', 'Illor','Tark', 'Corkes', 'Jeremy', 'Miriam', 'Elmore', 
               'Leena', 'Eovacius','Astra', 'Andal', 'Manfred', 'Casmetra', 'Zilare', 'Spint', 'Dargem']

towns = ["castle", "rampart", "tower", "inferno", "necropolis", "dungeon", "stronghold", "fortress", "conflux", "cove"]
heroes = {"castle": castle_heroes, "rampart": rampart_heroes, "tower": tower_heroes, "inferno": inferno_heroes, 
          "necropolis": necropolis_heroes, "dungeon": dungeon_heroes, "stronghold": stronghold_heroes, 
          "fortress": fortress_heroes, "conflux": conflux_heroes, "cove": cove_heroes}

In [4]:
if False:
    df2 = pd.DataFrame(columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", "turns", "template"])

    for row in df.iterrows():
        row = row[1]
        if row[1] > row[3]: result = 1
        elif row[1] < row[3]: result = 0
        else: result = 0.5
        df_temp = pd.DataFrame([[result, row[12], row[6], row[13], row[8] , row[14], row[7], row[11], row[5]]], 
                              columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", "turns", "template"])
        df2 = pd.concat([df2, df_temp])

        df_temp = pd.DataFrame([[1-result, row[14], row[7], {"red": "blue", "blue": "red", "white": "white"}[row[13]], 
                                 -row[8], row[12], row[6],row[11], row[5]]], 
                               columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", "turns", "template"])
        df2 = pd.concat([df2, df_temp])

    df2.to_csv("set2.csv", index = False)

In [56]:
df2 = pd.read_csv("set2.csv")

In [57]:
default = {}
for town in towns:
    subset = df2[(df2["town"] == town) & (df2["opponent_town"] == town)]
    default[town] = subset["bidding"].abs().mean()

res = {}
for town in towns:
    for op_town in towns:
        subset = df2[(df2["town"] == town) & (df2["opponent_town"] == op_town)]
        res[town+op_town] = subset["bidding"].abs().mean() - default[town]
    
print(res)

{'castlecastle': 0.0, 'castlerampart': -414.9246777449914, 'castletower': 287.50160071712116, 'castleinferno': 453.3495539334954, 'castlenecropolis': 1020.0162206001623, 'castledungeon': 203.66701425095562, 'castlestronghold': 737.1404036066979, 'castlefortress': 283.45056403450553, 'castleconflux': 691.2128017967434, 'castlecove': 16.296413836877036, 'rampartcastle': 18.175448368051775, 'rampartrampart': 0.0, 'ramparttower': 661.6662525512465, 'rampartinferno': 638.2552356020942, 'rampartnecropolis': 1555.3385689354277, 'rampartdungeon': 96.0403233213924, 'rampartstronghold': 1862.493040480143, 'rampartfortress': 828.3015318983903, 'rampartconflux': 665.3385689354277, 'rampartcove': 511.1927356020942, 'towercastle': -20.431286549707693, 'towerrampart': -79.3667608286255, 'towertower': 0.0, 'towerinferno': -194.92521367521385, 'towernecropolis': 943.6038011695905, 'towerdungeon': 2.510683760683605, 'towerstronghold': -200.05341880341894, 'towerfortress': -2.6175213675214763, 'towerconf

In [58]:
def town_v_town_winrate_heatmap(df):
    res = []
    for town in towns:
        res.append([])
        for op_town in towns[::-1]:
            subset = df[(df["town"] == town) & (df["opponent_town"] == op_town)]
            if len(subset) <= 16:
                winrate = None
            else:
                winrate = round(sum(subset["result"])/len(subset), 2)
            res[-1].append(winrate)
    
    return res

def town_v_town_bidding_heatmap(df):
    res = []
    for town in towns:
        res.append([])
        for op_town in towns[::-1]:
            subset = df[(df["town"] == town) & (df["opponent_town"] == op_town)]
            if len(subset) <= 16:
                bidding = None
            else:
                winrate = round(sum(subset["bidding"])/len(subset))
            res[-1].append(winrate)
    
    return res

def town_v_town_bidding_variance_heatmap(df):
    res = []
    for town in towns:
        res.append([])
        for op_town in towns[::-1]:
            subset = df[(df["town"] == town) & (df["opponent_town"] == op_town)]
            if len(subset) <= 16:
                bidding = None
            else:
                winrate = round(stdev(subset["bidding"]))
            res[-1].append(winrate)
    
    return res

In [59]:
# Town vs Town heatmap (Section two)

app = dash.Dash(__name__)
res_winrate = town_v_town_winrate_heatmap(df2)
res_bidding = town_v_town_bidding_heatmap(df2)
res_bidding_variance = town_v_town_bidding_variance_heatmap(df2)

fig_winrate = px.imshow(res_winrate, text_auto=True, labels = {"x": "Opponent Town", "y": "Player Town"}, 
                        x = towns[::-1], y = towns)

fig_bidding = px.imshow(res_bidding, text_auto=True, labels = {"x": "Opponent Town", "y": "Player Town"}, 
                        x = towns[::-1], y = towns)

fig_bidding_variance = px.imshow(res_bidding_variance, text_auto=True, labels = {"x": "Opponent Town", "y": "Player Town"}, 
                        x = towns[::-1], y = towns)

app.layout = html.Div([dcc.Graph(id="town_V_town_graph"),
                       dcc.Checklist(options = ["bidding", "winrate", "bidding variance"], value = ["bidding"], 
                                     id="town_V_town_check"),
                       dcc.Store(data = [], id="town_V_town_state")])

@app.callback(
    Output("town_V_town_graph", "figure"),
    Output("town_V_town_check", "value"),
    Output("town_V_town_state", "data"),
    Input("town_V_town_check", "value"),
    Input("town_V_town_state", "data"))
def town_V_town(value, state):
    value = list(set(value) - set(state))
    
    if value == ["bidding"] or (value == [] and state == ["bidding"]):
        return fig_bidding, ["bidding"], ["bidding"]
    
    elif value == ["winrate"] or (value == [] and state == ["winrate"]):
        return fig_winrate, ["winrate"], ["winrate"]
    
    elif value == ["bidding variance"] or (value == [] and state == ["bidding variance"]):
        return fig_bidding_variance, ["bidding variance"], ["bidding variance"]

In [87]:
def town_bar_graphs(sub_df):
    bidding_quantiles = []
    turns_quantiles = []
    for i in [0, 0.2, 0.4, 0.6, 0.8, 1]:
        bidding_quantiles.append(sub_df["bidding"].abs().quantile(i))
        turns_quantiles.append(sub_df["turns"].abs().quantile(i))

    for i in range(5):
        sub_df.loc[(sub_df["bidding"] >= bidding_quantiles[i]) & (sub_df["bidding"] > 0) & 
                   (sub_df["bidding"] <= bidding_quantiles[i+1]), "bidding quantiles"] = \
        f"{round(0.2*i,1)}-{round(0.2*(i+1),1)}"

        sub_df.loc[(sub_df["turns"] >= turns_quantiles[i]) & (sub_df["color"] == "red") & 
                   (sub_df["turns"] <= turns_quantiles[i+1]), "turns quantiles"] = \
        f"{round(0.2*i,1)}-{round(0.2*(i+1),1)}"

    sub_df.loc[sub_df["bidding"] <= 0]["bidding quantiles"] = np.nan
    sub_df.loc[sub_df["color"] == "blue"]["turns quantiles"] = np.nan

    bidding_x = []
    turns_x = []
    for i in ["0.0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"]:
        bidding_x.append(round(sub_df[sub_df["bidding quantiles"] == i]["result"].mean(), 2))
        turns_x.append(round(sub_df[sub_df["turns quantiles"] == i]["result"].mean(), 2))
    
    return bidding_x, turns_x

In [88]:
app = dash.Dash(__name__)


app.layout = html.Div([dcc.Graph(id="town_graph"),
                       dcc.Checklist(options = ["bidding", "turns"], value = ["bidding"], 
                                     id="town_check"),
                       dcc.Dropdown(towns, value = towns[0], id='town_dropdown'),
                       dcc.Store(data = [], id="town_state")])

@app.callback(
    Output("town_graph", "figure"),
    Output("town_check", "value"),
    Output("town_state", "data"),
    Input("town_check", "value"),
    Input("town_state", "data"),
    Input("town_dropdown", "value"))
def town_V_town(value, state, town):
    sub_df = df2[df2["town"] == town]
    print(sub_df)

    value = list(set(value) - set(state))
    
    if value == ["bidding"] or (value == [] and state == ["bidding"]):
        bidding_x = town_bar_graphs(sub_df)[0]
        return px.bar(x=["0.0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"], y=bidding_x), ["bidding"], ["bidding"]
    
    elif value == ["turns"] or (value == [] and state == ["turns"]):
        turns_x = town_bar_graphs(sub_df)[1]
        return px.bar(x=["0.0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"], y=turns_x), ["turns"], ["turns"]

In [89]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Mar/2023 17:17:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 17:17:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 17:17:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 17:17:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 17:17:04] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 17:17:04] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\

      result    town        hero color  bidding opponent_town opponent_hero  \
26       0.0  castle     valeska   red    -3600       inferno          calh   
30       0.0  castle     valeska   red    -3000    necropolis         thant   
87       1.0  castle     valeska  blue     2200       dungeon        shakti   
114      0.0  castle     valeska   red    -3500      fortress        wystan   
124      1.0  castle     valeska   red    -3800       inferno          calh   
...      ...     ...         ...   ...      ...           ...           ...   
6736     1.0  castle     valeska  blue     2700       conflux          luna   
6742     0.0  castle     valeska  blue    -1200       dungeon        deemer   
6754     0.0  castle    cuthbert   red      800        castle    lord haart   
6755     1.0  castle  lord haart  blue     -800        castle      cuthbert   
6765     0.0  castle     valeska   red     5400    stronghold       tyraxor   

      turns   template bidding quantiles turns quan

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

      result     town      hero color  bidding opponent_town opponent_hero  \
2        0.0  rampart   giselle   red      900       rampart         aeris   
3        1.0  rampart     aeris  blue     -900       rampart       giselle   
11       0.0  rampart      ivor  blue      700    necropolis         thant   
17       1.0  rampart      ivor  blue    -1300         tower          aine   
20       0.0  rampart  coronius   red      900       rampart        alagar   
...      ...      ...       ...   ...      ...           ...           ...   
6718     1.0  rampart      ivor   red    -4700      fortress         tazar   
6723     0.0  rampart      ivor  blue     -100         tower        solmyr   
6734     0.0  rampart  coronius   red     -900       rampart        jenova   
6735     1.0  rampart    jenova  blue      900       rampart      coronius   
6750     0.0  rampart      ivor   red        0      fortress        wystan   

      turns     template bidding quantiles turns quantiles  
2 

...

      result      town    hero  color  bidding opponent_town opponent_hero  \
18       0.0  fortress   alkin    red     2800       inferno          calh   
25       1.0  fortress  wystan   blue    -6200       conflux       grindan   
28       0.0  fortress   andra  white     -900      fortress         alkin   
29       1.0  fortress   alkin  white      900      fortress         andra   
34       0.0  fortress  wystan    red    -4000         tower          aine   
...      ...       ...     ...    ...      ...           ...           ...   
6725     0.0  fortress  wystan   blue     4800    necropolis         thant   
6733     0.5  fortress  wystan   blue     1100         tower          aine   
6751     1.0  fortress  wystan   blue        0       rampart          ivor   
6768     0.0  fortress   alkin   blue     -400      fortress         tazar   
6769     1.0  fortress   tazar    red      400      fortress         alkin   

      turns     template bidding quantiles turns quantiles  
18

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

      result     town     hero color  bidding opponent_town opponent_hero  \
12       0.0  conflux     luna   red    -2900         tower          aine   
24       0.0  conflux  grindan   red     6200      fortress        wystan   
60       0.0  conflux   monere   red      800       conflux         pasis   
61       1.0  conflux    pasis  blue     -800       conflux        monere   
90       0.0  conflux  grindan   red     1200    necropolis         thant   
...      ...      ...      ...   ...      ...           ...           ...   
6726     1.0  conflux     luna   red        0       inferno          calh   
6737     0.0  conflux     luna   red    -2700        castle       valeska   
6752     0.0  conflux     luna  blue    -1900       inferno          calh   
6766     1.0  conflux    pasis  blue      800       conflux        gelare   
6767     0.0  conflux   gelare   red     -800       conflux         pasis   

      turns         template bidding quantiles turns quantiles  
12     14.

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

      result  town      hero color  bidding opponent_town opponent_hero  \
6        1.0  cove     leena   red     -900          cove        jeremy   
7        0.0  cove    jeremy  blue      900          cove         leena   
8        1.0  cove  eovacius   red      700          cove        anabel   
9        0.0  cove    anabel  blue     -700          cove      eovacius   
22       0.0  cove    anabel   red    -1800       dungeon        shakti   
...      ...   ...       ...   ...      ...           ...           ...   
6731     1.0  cove    anabel   red    -3500         tower          iona   
6738     1.0  cove     andal   red     -900          cove      eovacius   
6739     0.0  cove  eovacius  blue      900          cove         andal   
6740     1.0  cove    anabel  blue    -3100       dungeon       jeddite   
6761     0.0  cove    anabel  blue    -3600    stronghold       tyraxor   

      turns     template bidding quantiles turns quantiles  
6       9.0        h3dm1              